In [18]:
import tensorflow as tf
import pandas as pd
import warnings; warnings.simplefilter('ignore')
import numpy as np
import warnings
import os
import sys
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.data.read_parallel import read_parallel_local

In [3]:
df = pd.read_csv("temp_vn.csv", encoding="latin1", parse_dates=True)
print(len(df))
df = df[~df.id.isna()]
df.id = df.id.astype(int)

# Sample a random version of each bill, at the end we only want
# one version of the bill in the dataset.
df_one_bill = df[['bill_id', 'version_number']].groupby('bill_id').sample(n=1, random_state=0)
df = df_one_bill.merge(df[['id', 'version_number', 'bill_id', 'partisan_lean', 'sc_id', 'signed']], on = ['version_number', 'bill_id'])

print(len(df))
df = df.sample(n=int(len(df)/40.0))
print(len(df))


df['text'] = read_parallel_local(df['id'], "../data/clean/")
df = df.reset_index(drop=True)

485388
199451
4986
Took 0.5167868812878926 min to open 4986 files with 20 processes.


In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [24]:
vocab_size = 10000
embedding_dim = 16
max_length = 1024
trunc_type = 'post'
padding_type = 'post'
oov_token = "<OOV>"

In [15]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['signed'], test_size=0.33, random_state=42)
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
corpus = X_train
tokenizer.fit_on_texts(corpus)

In [25]:
training_sequences = tokenizer.texts_to_sequences(X_train)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [26]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1024, 16)          160000    
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                12544     
_________________________________________________________________
dense (Dense)                (None, 24)                1560      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 174,129
Trainable params: 174,129
Non-trainable params: 0
_________________________________________________________________


In [27]:
num_epochs = 50
training_padded = np.array(training_padded)
training_labels = np.array(y_train)
testing_padded = np.array(testing_padded)
testing_labels = np.array(y_test)
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=1)

Epoch 1/50
105/105 [==============================] - 22s 182ms/step - loss: 0.5399 - accuracy: 0.8421 - val_loss: 0.4173 - val_accuracy: 0.8275
Epoch 2/50
105/105 [==============================] - 19s 178ms/step - loss: 0.3430 - accuracy: 0.8538 - val_loss: 0.3326 - val_accuracy: 0.8518
Epoch 3/50
105/105 [==============================] - 19s 179ms/step - loss: 0.2491 - accuracy: 0.9031 - val_loss: 0.3205 - val_accuracy: 0.8645
Epoch 4/50
105/105 [==============================] - 19s 177ms/step - loss: 0.2037 - accuracy: 0.9285 - val_loss: 0.3295 - val_accuracy: 0.8481
Epoch 5/50
105/105 [==============================] - 19s 178ms/step - loss: 0.1414 - accuracy: 0.9490 - val_loss: 0.3772 - val_accuracy: 0.8335
Epoch 6/50
105/105 [==============================] - 18s 175ms/step - loss: 0.1167 - accuracy: 0.9633 - val_loss: 0.4092 - val_accuracy: 0.8426
Epoch 7/50
105/105 [==============================] - 19s 178ms/step - loss: 0.0850 - accuracy: 0.9697 - val_loss: 0.4539 - val_ac

KeyboardInterrupt: 

import tensorflow as tf
import tensorflow as tf
import pandas as pd
import warnings; warnings.simplefilter('ignore')
import numpy as np
import warnings
import os
import sys
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.data.read_parallel import read_parallel_local
df = pd.read_csv("temp_vn.csv", encoding="latin1", parse_dates=True)
print(len(df))
df = df[~df.id.isna()]
df.id = df.id.astype(int)

# Sample a random version of each bill, at the end we only want
# one version of the bill in the dataset.
df_one_bill = df[['bill_id', 'version_number']].groupby('bill_id').sample(n=1, random_state=0)
df = df_one_bill.merge(df[['id', 'version_number', 'bill_id', 'partisan_lean', 'sc_id', 'signed']], on = ['version_number', 'bill_id'])

print(len(df))
df = df.sample(n=int(len(df)/40.0))
print(len(df))


df['text'] = read_parallel_local(df['id'], "../dat